# Baseline II: Facebook Prophet


In [1]:
from datetime import timedelta, datetime
import itertools
import json
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import os
import pandas as pd
from pathlib import Path
from time import time
import mlflow
import yaml

# Get the current project path (where you open the notebook)
# and go up two levels to get the project path
current_dir = Path.cwd()
proj_path = current_dir.parent

# make the code in src available to import in this notebook
import sys

sys.path.append(os.path.join(proj_path, "src"))

from fbprophet import Prophet

from metrics import *
from utils import *

# Catalog contains all the paths related to datasets
with open(os.path.join(proj_path, "conf/catalog.yml"), "r") as f:
    catalog = yaml.safe_load(f)["olist"]

# Params contains all of the dataset creation parameters and model parameters
with open(os.path.join(proj_path, "conf/params.yml"), "r") as f:
    params = yaml.safe_load(f)

# Experiment

In [4]:
# Step 1: Read data
merged_data = pd.read_csv(
    os.path.join(
        proj_path, catalog["output_dir"]["dir"], catalog["output_dir"]["transactions"]
    )
)

merged_data["order_approved_at"] = pd.to_datetime(merged_data["order_approved_at"])
# merged_data['order_approved_at'] = merged_data['order_approved_at'] + timedelta(days=3)

# Step2: Create date folds
date_ranges = make_dates(params["olist"]["experiment_dates"])

for prod_cat in params["olist"]["product_categories"]:
    print(f"Processing product category: {prod_cat}")

    # Initialize mlflow tracking
    create_folder(os.path.join(proj_path, "mlruns"))
    # mlflow.set_tracking_uri(os.path.join(proj_path, 'mlruns'))
    mlflow.set_tracking_uri(os.path.join("../../", "mlruns"))
    mlflow.set_experiment(prod_cat)
    metrics = []
    start_timer = time()
    all_predictions = []
    # Iterate over each period, unpack tuple in each variable.
    # in each of the period, we will find the best set of parameters,
    # which will represent the time-series cross validation methodology
    for (
        _,
        train_start,
        train_end,
        valid_start,
        valid_end,
        test_start,
        test_end,
    ) in date_ranges.itertuples():
        print(f"Processing range {str(train_start.date())} to {str(test_end.date())}")

        train_x = merged_data[
            (merged_data["order_approved_at"] >= train_start)
            & (merged_data["order_approved_at"] <= valid_end)
            & (merged_data["product_category_name"] == prod_cat)
        ][["order_approved_at", "payment_value"]]

        # Doesn't need a validation period.
        valid_x = merged_data[
            (merged_data["order_approved_at"] >= valid_start)
            & (merged_data["order_approved_at"] <= valid_end)
        ]
        test_y = merged_data[
            (merged_data["order_approved_at"] >= test_start)
            & (merged_data["order_approved_at"] <= test_end)
            & (merged_data["product_category_name"] == prod_cat)
        ][["order_approved_at", "payment_value"]]
        # Prophet expects two columns, one with the label 'ds' for the dates and y for the values
        train_x = train_x.rename(
            columns={"order_approved_at": "ds", "payment_value": "y"}
        )
        test_y = test_y.rename(
            columns={"order_approved_at": "ds", "payment_value": "y"}
        )

        # Iterate over the periods to make next-day forecasts
        predictions = []
        for i in range(test_y.shape[0]):

            # Instantiate a new Prophet object that represents the model
            model = Prophet(
                weekly_seasonality=True,
                yearly_seasonality=True,
                daily_seasonality=False,
            )

            # Call the built-in holiday collection for US to be included in the model
            model.add_country_holidays(country_name="BR")

            # Fit the FB Prohpet Model
            model.fit(pd.concat([train_x.iloc[i:], test_y.iloc[:i]]))
            future = model.make_future_dataframe(periods=1, freq="7D")
            fcst = model.predict(future)["yhat"].iloc[-1]
            predictions.append(fcst)

        all_predictions.extend(predictions)

    df_filtered = merged_data[
        (merged_data["product_category_name"] == prod_cat)
        & (
            merged_data["order_approved_at"]
            >= params["olist"]["experiment_dates"]["test_start"]
        )
        & (
            merged_data["order_approved_at"]
            <= params["olist"]["experiment_dates"]["test_end"]
        )
    ].copy()

    metrics = get_metrics(df_filtered["payment_value"].values, all_predictions)

    # store predictions
    fdir = os.path.join(proj_path, catalog["results"]["dir"])
    fname = os.path.join(fdir, f"exp1_prophet_{prod_cat}.csv")
    create_folder(fdir)

    save_data = pd.DataFrame(
        {
            "y_true": df_filtered["payment_value"].values,
            "preds": all_predictions,
            "dates": df_filtered["order_approved_at"],
        }
    )

    save_data.to_csv(fname)
    duration_min = int((time() - start_timer) // 60)

    with mlflow.start_run():
        mlflow.log_artifact(fname)
        mlflow.log_param("Product Category", prod_cat)
        mlflow.log_param("model", "prophet")
        mlflow.log_metrics(metrics)
        mlflow.log_metric("time", duration_min)

Processing product category: bed_bath_table
Processing range 2017-01-01 to 2018-01-28

Initial log joint probability = -108.839
Iteration  1. Log joint probability =     40.591. Improved by 149.43.
Iteration  2. Log joint probability =    80.8127. Improved by 40.2216.
Iteration  3. Log joint probability =    106.618. Improved by 25.8049.
Iteration  4. Log joint probability =    109.782. Improved by 3.16434.
Iteration  5. Log joint probability =    111.915. Improved by 2.1329.
Iteration  6. Log joint probability =    112.325. Improved by 0.410093.
Iteration  7. Log joint probability =    112.985. Improved by 0.660461.
Iteration  8. Log joint probability =     113.03. Improved by 0.044629.
Iteration  9. Log joint probability =    113.057. Improved by 0.0268259.
Iteration 10. Log joint probability =     113.06. Improved by 0.00334058.
Iteration 11. Log joint probability =     113.06. Improved by 5.47931e-05.
Iteration 12. Log joint probability =    113.061. Improved by 0.000295945.
Iterat

/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:401: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more d

4. Improved by 0.000155878.
Iteration 28. Log joint probability =    113.074. Improved by 3.37896e-05.
Iteration 29. Log joint probability =    113.074. Improved by 8.55933e-05.
Iteration 30. Log joint probability =    113.074. Improved by 5.33668e-05.
Iteration 31. Log joint probability =    113.074. Improved by 1.9677e-05.
Iteration 32. Log joint probability =    113.074. Improved by 4.91842e-07.
Iteration 33. Log joint probability =    113.074. Improved by 1.53931e-05.
Iteration 34. Log joint probability =    113.074. Improved by 3.8135e-06.
Iteration 35. Log joint probability =    113.074. Improved by 2.34721e-06.
Iteration 36. Log joint probability =    113.074. Improved by 3.63196e-07.
Iteration 37. Log joint probability =    113.074. Improved by 1.51197e-06.
Iteration 38. Log joint probability =    113.074. Improved by 6.05206e-07.
Iteration 39. Log joint probability =    113.074. Improved by 1.10548e-07.
Iteration 40. Log joint probability =    113.074. Improved by 3.4427e-07.


/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/a


Initial log joint probability = -108.391
Iteration  1. Log joint probability =    46.3863. Improved by 154.778.
Iteration  2. Log joint probability =    69.2302. Improved by 22.8439.
Iteration  3. Log joint probability =    103.807. Improved by 34.577.
Iteration  4. Log joint probability =    111.976. Improved by 8.16842.
Iteration  5. Log joint probability =    112.481. Improved by 0.505555.
Iteration  6. Log joint probability =    113.092. Improved by 0.610521.
Iteration  7. Log joint probability =    113.683. Improved by 0.591068.
Iteration  8. Log joint probability =    113.693. Improved by 0.010089.
Iteration  9. Log joint probability =    113.719. Improved by 0.0258437.
Iteration 10. Log joint probability =    113.754. Improved by 0.0352404.
Iteration 11. Log joint probability =    113.761. Improved by 0.00672566.
Iteration 12. Log joint probability =    113.786. Improved by 0.0249206.
Iteration 13. Log joint probability =    113.796. Improved by 0.00990505.
Iteration 14. Log jo

...

/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/a


Initial log joint probability = -108.761
Iteration  1. Log joint probability =    44.0994. Improved by 152.86.
Iteration  2. Log joint probability =    85.4563. Improved by 41.3569.
Iteration  3. Log joint probability =    109.201. Improved by 23.7448.
Iteration  4. Log joint probability =    111.954. Improved by 2.75246.
Iteration  5. Log joint probability =    116.585. Improved by 4.63183.
Iteration  6. Log joint probability =     116.78. Improved by 0.19446.
Iteration  7. Log joint probability =    116.995. Improved by 0.215282.
Iteration  8. Log joint probability =     117.24. Improved by 0.245222.
Iteration  9. Log joint probability =    117.266. Improved by 0.0252405.
Iteration 10. Log joint probability =    117.313. Improved by 0.0477955.
Iteration 11. Log joint probability =    117.349. Improved by 0.0357712.
Iteration 12. Log joint probability =    117.435. Improved by 0.0861102.
Iteration 13. Log joint probability =     117.45. Improved by 0.0149757.
Iteration 14. Log joint 

/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/a


Initial log joint probability = -109.286
Iteration  1. Log joint probability =    41.9086. Improved by 151.195.
Iteration  2. Log joint probability =    76.2134. Improved by 34.3048.
Iteration  3. Log joint probability =    102.168. Improved by 25.9549.
Iteration  4. Log joint probability =    112.569. Improved by 10.4007.
Iteration  5. Log joint probability =    116.747. Improved by 4.17768.
Iteration  6. Log joint probability =    117.856. Improved by 1.10962.
Iteration  7. Log joint probability =    117.859. Improved by 0.0028508.
Iteration  8. Log joint probability =    117.895. Improved by 0.0361706.
Iteration  9. Log joint probability =    117.895. Improved by 1.18061e-05.
Iteration 10. Log joint probability =      117.9. Improved by 0.00507413.
Iteration 11. Log joint probability =    117.904. Improved by 0.00349137.
Iteration 12. Log joint probability =    117.905. Improved by 0.00126983.
Iteration 13. Log joint probability =    117.907. Improved by 0.00165169.
Iteration 14. L

/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)
/home/abir/anaconda3/envs/ForecastingSales/lib/python3.7/site-packages/fbprophet/forecaster.py:434: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  .astype(np.float)


In [1]:
print(metrics)

NameError: name 'get_metrics' is not defined